In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import svm, feature_selection, linear_model, model_selection, metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import matplotlib.pylab as plb
from sklearn.exceptions import ConvergenceWarning
import warnings
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

In [2]:
# nrows can be adjusted bigger later
df = pd.read_csv('Manhattan.csv', skiprows=4)

In [3]:
df = df.rename(columns={'SALE\nPRICE':'SALE PRICE', 'APART\nMENT\nNUMBER':'APARTMENT NUMBER', 'EASE-MENT':'EASEMENT'})
df.drop(columns=['BOROUGH', 'EASEMENT', 'ADDRESS'], inplace=True)
df['TOTAL UNITS'] = df['TOTAL UNITS'].astype(str).str.replace(',', '')
df.astype({'TOTAL UNITS':'int64'})
df['SALE PRICE'] = pd.to_numeric(df['SALE PRICE'].str.replace(',', '', regex=True).str.replace('$','', regex=True))

In [4]:
# remove outliers based on z-score
def filter_outliers(df, col):
    z_scores = stats.zscore(df[col])
    # we define the threshold is 3 but exeute this procedure a reasonable times   
    zscore_threshold = 3
    mask = (abs(z_scores) < zscore_threshold)
    return df.loc[mask]

In [5]:
# we should execute this method many times until the mean value equal to median
for i in range(15):
    df = filter_outliers(df, 'SALE PRICE')

labelencode编码categorical value
use KNNImpute handle cells with missing value
use dummy handle categorical value

In [6]:
df.shape

(23459, 18)

In [7]:
categorical_cols = df.select_dtypes(include=['category', 'object']).columns

In [8]:
lb = LabelEncoder()
for col in categorical_cols:
    df[col] = lb.fit_transform(df[col].astype(str))

In [9]:
df.head(3)

,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
1,0,11,0,738,1307,0,0,10014,0,0,0,0,0,0,2,81,1654656,181
2,0,11,0,738,1308,0,0,10014,0,0,0,0,0,0,2,81,1069162,263
3,0,11,0,738,1309,0,0,10014,0,0,0,0,0,0,2,81,1374637,168


In [10]:
df.shape

(23459, 18)

In [11]:
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [12]:
df.shape

(23459, 18)

In [13]:
# check whether dataframe has some empty string cell or None values
def check_empty(df):
    for col in df.select_dtypes(include='object'):
        if df[col].str.strip().isna().any():
            print(f"Column {col} contains None values")
        if df[col].str.strip().eq('').any():
            print(f"Column {col} contains empty values")

In [14]:
check_empty(df)

In [15]:
df['SALE PRICE'].replace(0, np.nan, inplace=True)
# df['LAND SQUARE FEET'].replace(0, np.nan, inplace=True)
df.dropna(axis=0,  how='any', inplace=True)

In [16]:
df.shape

(15866, 18)

In [17]:
categorical_cols = df.select_dtypes(include=['category', 'object']).columns
df = pd.get_dummies(df, columns=categorical_cols)

In [18]:
def normalize(df):
    # select numerical columns
    num_cols=df.select_dtypes(include=[np.number]).copy()
    df_norm=((num_cols-num_cols.min())/(num_cols.max()-num_cols.min()))
    return df_norm

In [19]:
df = normalize(df)

In [20]:
df.shape

(15866, 18)

In [21]:
df.to_csv('big_clean_dataset.csv')